In [1]:
import pandas as pd
import requests
import threading
import queue
from itertools import cycle
from lxml.html import fromstring
from bs4 import BeautifulSoup
import time
from urllib.parse import quote

C:\Users\ibzcl\AppData\Local\Temp\ipykernel_16080\3876837736.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
root = "https://www.google.com/"
search_query = "Pakistan Travel"
results_limit = 1500

In [3]:
src = 'https://api64.ipify.org'
response = requests.get('https://free-proxy-list.net/')
parser = fromstring(response.text)
proxies = set()
usable_proxies = []

In [4]:
for i in parser.xpath('//tbody/tr'):
    if i.xpath('.//td[7][contains(text(),"yes")]'):
        proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
        proxies.add(proxy)

if proxies:
    proxy_pool = cycle(proxies)
    successful_responses = 0

    while successful_responses < 10:
        proxy = next(proxy_pool)
        try:
            r = requests.get(src, proxies={"http": proxy, "https": proxy}, timeout=10)
            r.raise_for_status()
            usable_proxies.append(proxy)
            successful_responses += 1
            print('Rotated IP %s succeed' % proxy)
            print('Total Successful IPs count: %d' % successful_responses)
        except Exception as e:
            print('Rotated IP %s failed (%s)' % (proxy, str(e)))

print("Total usable proxies:", len(usable_proxies))
print("Usable proxies:", usable_proxies)

Rotated IP 27.74.168.216:2019 succeed
Total Successful IPs count: 1
Rotated IP 178.128.113.118:23128 failed (HTTPSConnectionPool(host='api64.ipify.org', port=443): Max retries exceeded with url: / (Caused by ProxyError('Unable to connect to proxy', OSError('Tunnel connection failed: 502 Bad Gateway'))))
Rotated IP 67.43.227.227:4479 failed (HTTPSConnectionPool(host='api64.ipify.org', port=443): Max retries exceeded with url: / (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001EF9F9B7380>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))))
Rotated IP 189.240.60.163:9090 succeed
Total Successful IPs count: 2
Rotated IP 72.10.160.92:26077 failed (HTTPSConnectionPool(host='api64.ipify.org', port=443): Max retries exceeded with url: / (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPSCon

KeyboardInterrupt: 

In [13]:
def fetch_news(link, count=0, results=[]):
    if count >= results_limit:
        print("Collected 1500 news items. Stopping...")
        return pd.DataFrame(results)

    try:
        response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})

        if response.status_code == 429:
            print("Rate limit hit. Waiting for 5 seconds.")
            time.sleep(5)
            return fetch_news(link, count, results)

        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        for item in soup.find_all('div', attrs={'class': 'Gx5Zad fP1Qef xpd EtOod pkphOe'}):
            if count >= results_limit:
                print("Collected 1500 news items. Stopping...")
                return pd.DataFrame(results)

            raw_link = item.find('a', href=True)['href']
            news_link = raw_link.split("/url?q=")[1].split('&sa=U&')[0]

            title = item.find('div', attrs={'class': 'BNeawe vvjwJb AP7Wnd'}).get_text()
            full_description = item.find('div', attrs={'class': 'BNeawe s3v9rd AP7Wnd'}).get_text()

            if '·' in full_description:
                description, time_ago = full_description.rsplit(' · ', 1)
            else:
                description = full_description
                time_ago = "Time not specified"

            news_article = {
                'title': title,
                'description': description,
                'time_ago': time_ago,
                'link': news_link
            }

            print(f"Title: {title}")
            print(f"Link: {news_link}")
            print()

            results.append(news_article)
            count += 1
            time.sleep(8)

    except Exception as e:
        print(f"Error: {e}")
        time.sleep(10)
        return fetch_news(link, count, results)

    next_page = soup.find('a', attrs={'aria-label': 'Next page'})
    if next_page and count < results_limit:
        next_href = next_page['href']
        next_link = root + next_href
        return fetch_news(next_link, count, results)
    else:
        print("No more pages or reached 1500 news items.")
        return pd.DataFrame(results)


In [14]:
encoded_query = quote(search_query)
search_link = root + f"search?q={encoded_query}"

df = fetch_news(search_link)
print(df)
df.to_csv('scrappedData.csv')

Title: Pakistan travel - Lonely Planet | Asia
Link: https://www.lonelyplanet.com/pakistan
Title: Pakistan Tourism Development Corporation
Link: https://tourism.gov.pk/
Title: Pakistan International Travel Information
Link: https://travel.state.gov/content/travel/en/international-travel/International-Travel-Country-Information-Pages/Pakistan.html
Title: Pakistan Travel Advice & Safety | Smartraveller
Link: https://www.smartraveller.gov.au/destinations/asia/pakistan
Title: Travel advice and advisories for Pakistan - Travel.gc.ca
Link: https://travel.gc.ca/destinations/pakistan
Title: Pakistan Travel Advisory - Travel.gov
Link: https://travel.state.gov/content/travel/en/traveladvisories/traveladvisories/pakistan-travel-advisory.html
Title: Pakistan travel advice - GOV.UK
Link: https://www.gov.uk/foreign-travel-advice/pakistan
Title: Pakistan Travel Places: Pakistan Tourism Services
Link: https://pakistantravelplaces.com/
Title: Travel Pakistani
Link: https://travelpakistani.com/
Title: Pa